In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder

import jb_helper_functions_prep
from jb_helper_functions_prep import create_enc

In [2]:
df = pd.read_csv('athletes.csv')

In [3]:
df.dropna(inplace=True)

In [4]:
df.shape

(10858, 11)

In [6]:
create_enc(df, ['sport', 'nationality'])

,id,name,nationality,sex,dob,height,weight,sport,gold,silver,bronze,sport_enc,nationality_enc
0,736041664,A Jesus Garcia,ESP,male,10/17/69,1.72,64.0,athletics,0,0,0,2,59
1,532037425,A Lam Shin,KOR,female,9/23/86,1.68,56.0,fencing,0,0,0,8,101
2,435962603,Aaron Brown,CAN,male,5/27/92,1.98,79.0,athletics,0,0,1,2,33
3,521041435,Aaron Cook,MDA,male,1/2/91,1.83,80.0,taekwondo,0,0,0,21,117
4,33922579,Aaron Gate,NZL,male,11/26/90,1.81,71.0,cycling,0,0,0,6,138
5,173071782,Aaron Royle,AUS,male,1/26/90,1.80,67.0,triathlon,0,0,0,23,10
6,266237702,Aaron Russell,USA,male,6/4/93,2.05,98.0,volleyball,0,0,1,24,191
7,382571888,Aaron Younger,AUS,male,9/25/91,1.93,100.0,aquatics,0,0,0,0,10
8,87689776,Aauri Lorena Bokesa,ESP,female,12/14/88,1.80,62.0,athletics,0,0,0,2,59
9,997877719,Ababel Yeshaneh,ETH,female,7/22/91,1.65,54.0,athletics,0,0,0,2,61


In [46]:
def remove_outlier_column(df, groupby_column, outlier_column):
    uniques = list(sorted(df[groupby_column].unique()))
    
    removed_df = pd.DataFrame()
    final_df = pd.DataFrame()
    
    for unique in uniques:
        df_grouped_1 = df[df[groupby_column] == unique]
        df_grouped_2 = df[df[groupby_column] == unique]
        
        q1 = df[outlier_column].quantile(0.25)
        q3 = df[outlier_column].quantile(0.75)
    
        iqr = q3-q1
        fence_low = q1-1.5*iqr
        fence_high = q3+1.5*iqr
        
        df_rem = df_grouped_1.loc[(df_grouped_1[outlier_column] < fence_low) | (df_grouped_1[outlier_column] > fence_high)]
        removed_df = removed_df.append(df_rem)
        
        df_out = df_grouped_2.loc[(df_grouped_2[outlier_column] > fence_low) & (df_grouped_2[outlier_column] < fence_high)]
        final_df = final_df.append(df_out)
        
    return final_df, removed_df

In [47]:
df_good, df_rem = remove_outlier_column(df, 'sex', 'height')

In [48]:
df_good.shape

(10766, 13)

In [49]:
df_rem.shape

(92, 13)

In [40]:
df_good.describe()

,id,height,weight,gold,silver,bronze,sport_enc,nationality_enc
count,1.076600e+04,10766.000000,10766.000000,10766.000000,10766.000000,10766.000000,10766.000000,10766.000000
mean,5.009902e+08,1.765712,71.914453,0.059168,0.058332,0.058146,9.683448,92.538176
std,2.908979e+08,0.109172,15.911793,0.257775,0.242178,0.238356,8.189461,57.891926
min,1.834700e+04,1.470000,35.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.459458e+08,1.690000,60.000000,0.000000,0.000000,0.000000,2.000000,45.000000
50%,5.021887e+08,1.760000,70.000000,0.000000,0.000000,0.000000,8.000000,82.000000
75%,7.549556e+08,1.840000,80.000000,0.000000,0.000000,0.000000,16.000000,148.000000
max,9.999878e+08,2.060000,170.000000,5.000000,2.000000,2.000000,26.000000,199.000000


In [41]:
df.describe()

,id,height,weight,gold,silver,bronze,sport_enc,nationality_enc
count,1.085800e+04,10858.000000,10858.000000,10858.000000,10858.000000,10858.000000,10858.000000,10858.000000
mean,5.011329e+08,1.766942,72.070915,0.059772,0.058482,0.058574,9.688893,92.448241
std,2.909399e+08,0.113111,16.162358,0.260756,0.242387,0.239113,8.191945,57.862221
min,1.834700e+04,1.210000,31.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.460081e+08,1.690000,60.000000,0.000000,0.000000,0.000000,2.000000,44.000000
50%,5.020973e+08,1.760000,70.000000,0.000000,0.000000,0.000000,8.000000,82.000000
75%,7.555681e+08,1.840000,81.000000,0.000000,0.000000,0.000000,16.000000,148.000000
max,9.999878e+08,2.210000,170.000000,5.000000,2.000000,2.000000,26.000000,199.000000


In [51]:
df_rem.describe()

,id,height,weight,gold,silver,bronze,sport_enc,nationality_enc
count,9.200000e+01,92.000000,92.000000,92.000000,92.000000,92.000000,92.000000,92.000000
mean,5.178273e+08,1.910870,90.380435,0.130435,0.076087,0.108696,10.326087,81.923913
std,2.969513e+08,0.308846,29.546329,0.496524,0.266590,0.312963,8.499656,53.526872
min,3.094916e+07,1.210000,31.000000,0.000000,0.000000,0.000000,0.000000,6.000000
25%,2.775292e+08,1.457500,58.000000,0.000000,0.000000,0.000000,4.000000,38.000000
50%,5.012866e+08,2.080000,101.500000,0.000000,0.000000,0.000000,4.000000,64.000000
75%,7.772501e+08,2.100000,113.000000,0.000000,0.000000,0.000000,16.000000,132.500000
max,9.991311e+08,2.210000,130.000000,4.000000,1.000000,1.000000,25.000000,191.000000


In [52]:
df_good, df_rem = remove_outlier_column(df_good, 'sex', 'weight')

In [53]:
df_good.shape

(10526, 13)

In [54]:
df_rem.shape

(207, 13)

### This is set up so that you can run a single column you would like to group by and eliminate outliers in a single column.  If you use df_good, for example, you would have to run that back through the second round, with weight, like above.